In [2]:
import tensorflow
import nltk
import json
import codecs
import csv
import regex as re
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import keras
from keras.layers import SimpleRNN
from keras.preprocessing.sequence import pad_sequences

Readine the lines and the conversation texts to extract text for training the chatbot

In [3]:
# Open the movie_lines file and econding used for this file is utf-8, read function helps in reading the file and 
#split function formats the data
lines = open('movie_lines.txt',encoding='utf-8', errors='ignore').read().split('\n')

#Reading the conversations file
conv_lines = open('movie_conversations.txt',encoding='utf-8',errors='ignore').read().split('\n')

In [4]:
#Extracting the id and the conversation from the movie_lines text file.
# Step 1 - Splitting the file on the indicator '+++$+++'
# Step 2 - Extracting the 5th section of each line as that depicts  the conversation and storing the convesation against the ID
#          present in the first section of the sentence
id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]

# Conversation lines data 
# Step 1: Splitting each line on the identifier
# Step 2: Extracting the last segement of the sentence and then replace quotation marks and in between spaces
conv=[]
for line in conv_lines:
    _conv = line.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
    conv.append(_conv.split(','))

In [5]:
#A sample conversation -
#u16 +++$+++ u25 +++$+++ m1 +++$+++ ['L2256', 'L2257', 'L2258', 'L2259', 'L2260'] this is the 250th row from the input
for i in conv[0]:
    print(i, id2line[i])

L194 Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.
L195 Well, I thought we'd start with pronunciation, if that's okay with you.
L196 Not the hacking and gagging and spitting part.  Please.
L197 Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?


In [6]:
# Taking a pair of conversation and breaking it into input and response pair, ny drawing the lines from the id2line dict
pairs=[]
for cnv in conv:
    for i in range(len(cnv)-1):
        inp=id2line[cnv[i]].strip()
        res=id2line[cnv[i+1]].strip()
        if inp and res:
            pairs.append([inp,res])

In [7]:
pairs[0:10]

[['Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.',
  "Well, I thought we'd start with pronunciation, if that's okay with you."],
 ["Well, I thought we'd start with pronunciation, if that's okay with you.",
  'Not the hacking and gagging and spitting part.  Please.'],
 ['Not the hacking and gagging and spitting part.  Please.',
  "Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?"],
 ["You're asking me out.  That's so cute. What's your name again?",
  'Forget it.'],
 ["No, no, it's my fault -- we didn't have a proper introduction ---",
  'Cameron.'],
 ['Cameron.',
  "The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does."],
 ["The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.",
  'Seems like she could get a date easy enough...'],
 [

In [8]:
#Storing inputs and response pair in a text file
file_name='conv_formatted.txt'
delimiter = '\t'
# Unescape the delimiter
delimiter = str(codecs.decode(delimiter, "unicode_escape"))
with open(file_name,'w',encoding='utf-8') as out_file:
    writer=csv.writer(out_file,delimiter=delimiter, lineterminator='\n')
    for pair in pairs:
        writer.writerow(pair)

In [9]:
#Reading the formatted conversation file
line_fmt = open(file_name,encoding='utf-8').read().strip().split('\n')       #Strip removes the leading and trailing characters
line_fmt[0]

"Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\tWell, I thought we'd start with pronunciation, if that's okay with you."

In [10]:
#Some basic formatting of the data
#Converting the data into lower-case, trim and remove all non-letter characters
def NormalizeText(s):
    s=s.lower().strip()
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

#filterPairs - Stripping sentences into smaller ones by setting a threshold limit on number of words, if either the input
#or the response is less than threshold length, then it is added to the list of valid pairs, else skipped
#'Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.
#Again.\tWell, I thought we'd start with pronunciation, if that's okay with you.'
#the above sentence pair is not considered as valid when the max_length is 10 as both the input & response's length > 10
def filterPairs(pairs,max_length):
    valid_pair=[]
    for pair in pairs:
        inp, resp = pair[0].split(' '),pair[1].split(' ')
        if len(inp) < max_length and len(resp) < max_length:
            valid_pair.append(pair)
    print(f'load total {len(valid_pair)} pairs with length <= max_length (10)')
    return valid_pair


def clean_text(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|]", "", text)
#     text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    text = " ".join(text.split())
    return text

pairs=[[NormalizeText(clean_text(s)) for s in l.split('\t')] for l in line_fmt]
valid_pairs=filterPairs(pairs,10)
print(valid_pairs[0:3])

load total 65049 pairs with length <= max_length (10)
[['there .', 'where ?'], ['you have my word . as a gentleman', 'you are sweet .'], ['hi .', 'looks like things worked out tonight huh ?']]


The first thing to do is to create values for our start of sentence, end of sentence, and sentence padding special tokens. When we tokenize text (split text into its atomic constituent pieces), we need special tokens to delineate both the beginning and end of a sentence, as well as to pad sentence (or some other text chunk) storage structures when sentences are shorter then the maximum allowable space. More on this later.

PAD_token = 0   # Used for padding short sentences

SOS_token = 1   # Start-of-sentence token

EOS_token = 2   # End-of-sentence token

word2index - assigns an index to each unique word

word2count - Measures the frequency of each word in the corpus. Example - 'there': 2013, the word there appears 2013 times

index2word - Used for retrieving the word after predicting

In [11]:
pad_token=0
sos_token=1
eos_token=2

class Vocab:
    
    def __init__(self):
        self.trimmed = False
        self.word2count={}
        self.word2index={}
        self.index2word={pad_token:'PAD',sos_token:'SOS',eos_token:'EOS'}
        self.numword=3
        
    def add_sentence(self, sentence):
        for word in sentence.split(' '):
            self.addword(word)
    
    def addword(self,word):
        if word not in self.word2index:
            self.word2index[word]=self.numword
            self.word2count[word]=1
            self.index2word[self.numword]=word
            self.numword+=1
        else:
            self.word2count[word]+=1
     
    def trim(self,min_count):
        if self.trimmed:
            return
        self.trimmed = True
        
        keep_word = []
        for word in self.word2count:
            if self.word2count[word] > min_count:
                keep_word.append(word)
        
        #Dictionary is reinitialized to setup the word2index dictionary
        self.word2count={}
        self.word2index={}
        self.index2word={pad_token:'PAD',sos_token:'SOS',eos_token:'EOS'}
        self.numword=3
        
        for word in keep_word:
            self.addword(word)

In [12]:
voc=Vocab()
for pair in valid_pairs:
    voc.add_sentence(pair[0])
    voc.add_sentence(pair[1])
    
print(f'total word in vocabulary is : {voc.numword}')

total word in vocabulary is : 19477


Trimming the rare words from the input pairs so that the model can converge easily as these rare words hinder the convergence of the model

Step 1 
Remove all the words having min_count less than 3 from vocabulary

Step 2
Used to vocabulary to remove pair of words which contains word that is not part of the dictionary

In [13]:
def trim_rareword(voc,valid_pairs,min_count):

    voc.trim(min_count)     #Rebuilding the word2index dictionary to trim the rarewords
    
    trimmed_pair = []
    for pair in valid_pairs:
        inp_sent = pair[0]
        res_sent = pair[1]
        keep_input = True
        keep_resp = True
        
        for word in inp_sent.split(' '):
            if word not in voc.word2index:
                keep_input = False
                break
        
        for word in res_sent.split(' '):
            if word not in voc.word2index:
                keep_resp = False
                break
            
        if keep_input & keep_resp:
            trimmed_pair.append(pair)
    print(f'the trimming process make the total {len(valid_pairs)} ==> {len(trimmed_pair)} trimmed pair)')
    return voc,trimmed_pair

In [14]:
voc,trimmed_pairs = trim_rareword(voc,valid_pairs,min_count = 20)

the trimming process make the total 65049 ==> 29857 trimmed pair)


In [15]:
voc.numword

1544

In [16]:
for pair in trimmed_pairs:
    for word1,word2 in zip(pair[0].split(' '),pair[1].split(' ')):
        if word1 not in voc.word2index:
            print(word1)
        if word2 not in voc.word2index:
            print(word2)

Data Transform to Tensor

In [17]:
trimmed_pairs[:6]

[['there .', 'where ?'],
 ['you have my word . as a gentleman', 'you are sweet .'],
 ['hi .', 'looks like things worked out tonight huh ?'],
 ['well no . . .', 'then that is all you had to say .'],
 ['then that is all you had to say .', 'but'],
 ['what good stuff ?', 'the real you .']]

In [18]:
#Converting the trimmed_pairs to vector representatiom
word2vec = [voc.word2index[word] for word in trimmed_pairs[5][0].split(' ')]

In [19]:
word2vec

[47, 48, 49, 6]

In [20]:
for pair in trimmed_pairs:
    inp_cnt=0
    resp_cnt=0
    for word in pair[0].split(' '):
        inp_cnt+=1
    for word in pair[1].split(' '):
        resp_cnt+=1
    if inp_cnt > 10 and resp_cnt > 10:
        print(pair)

In [21]:
def transform1(voc,sentence):
    return [voc.word2index[word] for word in sentence.split(' ')]+[eos_token]

In [22]:
max_line_length = 10
encoder_inp=[]
decoder_inp=[]
for pair in trimmed_pairs:
    encoder_inp.append(transform1(voc,pair[0]))
    decoder_inp.append([sos_token]+transform1(voc,pair[1]))    

In [23]:
def transform(voc, trimmed_pairs):

    encoder_inp = np.zeros(shape=(len(trimmed_pairs), 20))
    encoder_out = np.zeros(shape=(len(trimmed_pairs), 20))
    idx=0
    for pair in trimmed_pairs:
        
        j=0
        for word1,word2 in zip(pair[0].split(' '),pair[1].split(' ')):
            encoder_inp[idx][j]=voc.word2index[word1]
            encoder_out[idx][j]=voc.word2index[word2]
            j+=1
        encoder_inp[idx][j]=eos_token
        encoder_out[idx][j]=eos_token
        idx+=1
    
    return encoder_inp,encoder_out

In [24]:
#encoder_input,encoder_output=transform(voc,trimmed_pairs)

In [25]:
#print(encoder_input_data.shape,decoder_input_data.shape[:5])

First Model -
1. Embedding Layer for drawing insight from the text data
2. LSTM layer for Encoder
3. LSTM layer for Decoder
4. Dense Model

In [27]:
encoder_inputs = keras.layers.Input(shape=(None,),dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=(None,),dtype=np.int32)

with tf.device('/GPU:0'):
    encoder_embeddings = keras.layers.Embedding(voc.numword, 128)(encoder_inputs)
    decoder_embeddings = keras.layers.Embedding(voc.numword, 128)(decoder_inputs)    
    
    encoder = keras.layers.LSTM(512,return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_embeddings)
    encoder_state = [state_h, state_c]
    
    print(f'Encoder:',encoder_outputs)
    
    decoder_cell = keras.layers.LSTM(512,return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_cell(decoder_embeddings,initial_state=encoder_state)
    output_layer = keras.layers.Dense(voc.numword,activation='softmax')
    decoder_outputs = output_layer(decoder_outputs)

Encoder: KerasTensor(type_spec=TensorSpec(shape=(None, 512), dtype=tf.float32, name=None), name='lstm_1/PartitionedCall:0', description="created by layer 'lstm_1'")


In [29]:
with tf.device('/GPU:0'):
    
    encoder_input_data = pad_sequences(encoder_inp, maxlen=max_line_length+1, value=pad_token, padding='post')
    decoder_input_data = pad_sequences(decoder_inp,maxlen=max_line_length+2,value=pad_token,padding='post')
    decoder_target_data = np.zeros((len(decoder_input_data), max_line_length+2, voc.numword),dtype='float32')
    
    model = keras.Model([encoder_inputs,decoder_inputs],decoder_outputs)
    model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['acc'])
    model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, None, 128)    197632      ['input_3[0][0]']                
                                                                                                  
 embedding_3 (Embedding)        (None, None, 128)    197632      ['input_4[0][0]']                
                                                                                              

In [30]:
print(decoder_target_data.shape, encoder_input_data.shape, decoder_input_data.shape)

(29857, 12, 1544) (29857, 11) (29857, 12)


In [31]:
resp_data = []
for pair in trimmed_pairs:
    resp_data.append(pair[1])

In [32]:
for i, target_seq in enumerate(resp_data):
    for t, seq in enumerate(target_seq.split(' ')):
        if t > 0:
            decoder_target_data[i,t-1,voc.word2index[seq]]=1

In [33]:
#Training the model
with tf.device('/GPU:0'):
    model.fit([encoder_input_data,decoder_input_data],decoder_target_data,
              batch_size=128,
              epochs=40,
              validation_split=0.05)

Epoch 1/40
222/222 [==============================] - 10s 25ms/step - loss: 1.6262 - acc: 0.0673 - val_loss: 1.6610 - val_acc: 0.0631
Epoch 2/40
222/222 [==============================] - 4s 18ms/step - loss: 1.6235 - acc: 0.0676 - val_loss: 1.6424 - val_acc: 0.0631
Epoch 3/40
222/222 [==============================] - 4s 19ms/step - loss: 1.6178 - acc: 0.0676 - val_loss: 1.6407 - val_acc: 0.0631
Epoch 4/40
222/222 [==============================] - 4s 19ms/step - loss: 1.6156 - acc: 0.0675 - val_loss: 1.6387 - val_acc: 0.0631
Epoch 5/40
222/222 [==============================] - 4s 18ms/step - loss: 1.6161 - acc: 0.0676 - val_loss: 1.6476 - val_acc: 0.0631
Epoch 6/40
222/222 [==============================] - 4s 19ms/step - loss: 1.6152 - acc: 0.0676 - val_loss: 1.6460 - val_acc: 0.0631
Epoch 7/40
222/222 [==============================] - 4s 18ms/step - loss: 1.6144 - acc: 0.0676 - val_loss: 1.6433 - val_acc: 0.0631
Epoch 8/40
222/222 [==============================] - 4s 19ms/step -

In [40]:
model.save('C:/Users/manju/OneDrive/Desktop/Ruparna/ChatBot Using RNN/movie_dialogue_lstm_model.h5')